In [1]:
import os
import re
import cv2
import time
import datetime

import ephem
import pandas as pd
import multiprocessing as mp
from cargan.utils.IPCam import IPCam

import sys
from jinja2 import Environment, FileSystemLoader

DATASET = 'data.tsv'
DEFAULT_DIR = './IPCam'
DEFAULT_SEQ = 10

In [2]:
def has_day_light(row):
    o = ephem.Observer()
    o.lat = str(row['latitude'])
    o.long = str(row['longitude'])
    sun = ephem.Sun(o)
    return float(sun.alt)

def func(ip):
    camera = IPCam(ip['ip_cam'])
    parent_dir = '_'.join([re.sub(r'\W+', '', ip['city']) ,
                           re.sub(r'\W+', '', ip['code']),
                           str(ip['zip']).strip('-_')])

    sequence_dir = '_'.join(['day' if ip['current_lighting'] > 0.1 else 'night', 
                             datetime.datetime.now().strftime("%Y%m%d_%H%M%S")])
    output_dir = os.path.join(DEFAULT_DIR, parent_dir, sequence_dir)
    if not os.path.isdir(output_dir):
        try:
            os.makedirs(output_dir)
        except Exception as error:
            print(error)
            pass

    with open(os.path.join(output_dir, 'ip_address.txt'),'w') as fio:
        fio.write(ip['ip_cam'])
        
    try:
        start_time = time.time()
        frames = camera.get_sequence(DEFAULT_SEQ)
        [cv2.imwrite(os.path.join(output_dir,'%s.jpg' % frame_id), frame) 
         for frame_id, frame in enumerate(frames)]       
        print("--- Completed %s in  %s seconds ---" % (os.path.join(parent_dir, sequence_dir), 
                                                       time.time() - start_time))
        return {parent_dir: {
            ip['ip_cam']: [os.path.join(parent_dir, sequence_dir,'%s.jpg' % frame_id)
                            for frame_id in range(len(frames))]
        }}
    except Exception as error:
        os.rmdir(output_dir)
        print("Remove %s becase of %s" % (output_dir, error))
        pass

In [3]:
df = pd.read_csv(DATASET,sep='\t')
df['current_lighting'] = df.T.apply(lambda row: has_day_light(row))
_ = df.pop('Notes')
df = df.sort_values(by=['current_lighting'], axis=0,ascending=False)
df = df.reset_index(drop=True)
df = df[df.current_lighting >= 0.05]
len(df)

51

In [4]:
cpu = 4
pool = mp.Pool(cpu)
temp = df.T.to_dict()
results = pool.map(func,[temp[key] for key in temp])

--- Completed Anseong_KR_456-320/day_20180209_233008 in  7.69831418991 seconds ---
--- Completed Seoul_KR_100-101/day_20180209_233008 in  11.2769970894 seconds ---
--- Completed Seoul_KR_100-101/day_20180209_233016 in  10.4973909855 seconds ---
--- Completed Jakarta_ID_14310/day_20180209_233008 in  19.6783950329 seconds ---
--- Completed Seoul_KR_100-101/day_20180209_233019 in  12.3607752323 seconds ---
--- Completed Seoul_KR_100-101/day_20180209_233026 in  9.99733018875 seconds ---
--- Completed Taipei_TW_100/day_20180209_233008 in  31.0860202312 seconds ---
--- Completed Seoul_KR_100-101/day_20180209_233032 in  13.0347969532 seconds ---
--- Completed Seoul_KR_100-101/day_20180209_233036 in  11.0662438869 seconds ---
--- Completed Seoul_KR_100-101/day_20180209_233045 in  10.5256199837 seconds ---
--- Completed Seoul_KR_100-101/day_20180209_233047 in  10.8716719151 seconds ---
--- Completed Taipei_TW_100/day_20180209_233039 in  21.2213439941 seconds ---
--- Completed Seoul_KR_100-101/d

OSError: [Errno 39] Directory not empty: './IPCam/Tochigi_JP_328-0024/day_20180209_233313'

--- Completed Stockholm_SE_19587/night_20180209_233314 in  13.3432910442 seconds ---
--- Completed Stockholm_SE_19587/night_20180209_233327 in  18.4150769711 seconds ---


In [ ]:
env = Environment(loader=FileSystemLoader(os.path.join(sys.path[0], 'templates')))
template = env.get_template('overview.html')
parsed_template = template.render(videos=results)
with open(os.path.join(DEFAULT_DIR,'visualization.html'), "w") as fio:
    fio.write(parsed_template)

In [ ]:
results